In [1]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

In [3]:
%pip install -qU \
    replicate \
    langchain \
    sentence_transformers \
    pdf2image \
    pdfminer \
    pdfminer.six \
    unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 26.7 MB/s

In [4]:
pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.4 MB/s eta 0:00:00


In [5]:
pip install pypdf

In [6]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 11.2 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# langchain setup
from langchain.llms import Replicate
# Use the Llama 2 model hosted on Replicate
# Temperature: Adjusts randomness of outputs, greater than 1 is random and 0 is deterministic, 0.75 is a good starting value
# top_p: When decoding text, samples from the top p percentage of most likely tokens; lower to ignore less likely tokens
# max_new_tokens: Maximum number of tokens to generate. A word is generally 2-3 tokens


from getpass import getpass
import os

REPLICATE_API_TOKEN = "r8_TpNRKTh8GINVY78f3owhndEgoOp9WmB2CuHFX"
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN


llama2_13b = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

llama_model = Replicate(
     model=llama2_13b,
     model_kwargs={"temperature": 0.75,"top_p": 1, "max_new_tokens":1000}
 )

In [10]:
# Step 1: load the external data source. In our case, we will load Meta’s “Responsible Use Guide” pdf document.
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
#loader = OnlinePDFLoader("https://ai.meta.com/static-resource/responsible-use-guide/")
loader = PyPDFDirectoryLoader('./data')
documents = loader.load()

# Step 2: Get text splits from document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# Step 3: Use the embedding model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2" # embedding model
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Step 4: Use vector store to store embeddings
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [11]:
def md(t):
  display(Markdown(t))

In [12]:
# Query against your own data
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []
query = "what is covid 19 treatment in severe cases"
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Based on the information provided, there are several treatments for severe cases of COVID-19. These include:

1. Antiviral medications: These drugs can help reduce the severity of symptoms and the risk of complications. Examples of antiviral medications used to treat COVID-19 include remdesivirexternal icon and lopinavirexternal icon.
2. Corticosteroids: These medications can help reduce inflammation in the body and improve breathing. They may be given to patients with severe respiratory distress.
3. Oxygen therapy: This involves providing extra oxygen to the body through an oxygen mask or nasal tubes. It can help improve oxygen levels in the blood and relieve shortness of breath.
4. Ventilation support: Patients with severe COVID-19 may require mechanical ventilation to support their breathing. This can involve using a ventilator to take over the work of breathing for the patient.
5. Extracorporeal membrane oxygenation (ECMO): This is a type of life support that uses a machine to take over the work of the heart and lungs. It may be used in severe cases of COVID-19 where other treatments have not been effective.

It's important to note that these treatments should only be administered by trained medical professionals and under the guidance of a healthcare provider. Additionally, the best treatment approach will depend on the individual patient and their specific needs, so it's important to consult with a healthcare professional for personalized advice.